In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

from preprocess import get_cifar100_datasets

train_ds, val_ds, test_ds = get_cifar100_datasets(image_size=(128, 128), batch_size=32)

## Building ResNet-50

In [3]:
def identity_block(X, f, filters):
    X_shortcut=X

    X=tfl.Conv2D(filters=filters[0],kernel_size=1,strides=(1,1), padding='valid')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)
    X=tfl.Activation('relu')(X)

    X=tfl.Conv2D(filters=filters[1],kernel_size=f,strides=(1,1), padding='same')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)
    X=tfl.Activation('relu')(X)

    X=tfl.Conv2D(filters=filters[2],kernel_size=1,strides=(1,1), padding='valid')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)

    X=tfl.Add()([X_shortcut,X])
    X=tfl.Activation('relu')(X)

    return X


In [4]:
def convolutional_block(X, f, filters, s=2):
    X_shortcut=X

    X=tfl.Conv2D(filters=filters[0],kernel_size=1,strides=(s,s), padding='valid')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)
    X=tfl.Activation('relu')(X)

    X=tfl.Conv2D(filters=filters[1],kernel_size=f,strides=(1,1), padding='same')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)
    X=tfl.Activation('relu')(X)

    X=tfl.Conv2D(filters=filters[2],kernel_size=1,strides=(1,1), padding='valid')(X)
    X=tfl.BatchNormalization(axis=3)(X, training=True)

    X_shortcut=tfl.Conv2D(filters=filters[2],kernel_size=1,strides=(s,s), padding='valid')(X_shortcut)
    X_shortcut=tfl.BatchNormalization(axis=3)(X_shortcut, training=True)

    X=tfl.Add()([X_shortcut,X])
    X=tfl.Activation('relu')(X)

    return X

In [ ]:
def arch(input_shape):

    input_img = tf.keras.Input(shape=input_shape)

    layer =tfl.ZeroPadding2D((3, 3))(input_img)

    layer=tfl.Conv2D(filters=64,kernel_size=7,strides=(2,2))(layer)
    layer=tfl.BatchNormalization(axis=3)(layer, training=True)
    layer=tfl.Activation('relu')(layer)
    layer=tfl.MaxPooling2D((3, 3), strides=(2, 2))(layer)

    layer=convolutional_block(layer,3,[64,64,256],1)
    layer=identity_block(layer,3,[64,64,256])
    layer=identity_block(layer,3,[64,64,256])

    layer=convolutional_block(layer,3,[128,128,512],2)
    layer=identity_block(layer,3,[128,128,512])
    layer=identity_block(layer,3,[128,128,512])
    layer=identity_block(layer,3,[128,128,512])

    layer=convolutional_block(layer,3, [256, 256, 1024],2)
    layer=identity_block(layer,3, [256, 256, 1024])
    layer=identity_block(layer,3, [256, 256, 1024])
    layer=identity_block(layer,3, [256, 256, 1024])
    layer=identity_block(layer,3, [256, 256, 1024])
    layer=identity_block(layer,3, [256, 256, 1024])

    layer=convolutional_block(layer,3, [512, 512, 2048],2)
    layer=identity_block(layer,3, [512, 512, 2048])
    layer=identity_block(layer,3, [512, 512, 2048])

    layer=tfl.AveragePooling2D(pool_size=(2, 2),padding='same')(layer)
    layer=tfl.Flatten()(layer)

    outputs=tfl.Dense(units= 100 , activation='softmax')(layer)
    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    return model

## training and evaluating the model

In [6]:
conv_model = arch((128, 128, 3)) 
conv_model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
conv_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 64)   256         conv2d[0][0]                     
______________________________________________________________________________________________

In [7]:
# Train the model
history = conv_model.fit(
    train_ds,
    epochs=4,
    validation_data=val_ds
)

Epoch 1/4
1407/1407 [==============================] - 120s 75ms/step - loss: 4.2600 - accuracy: 0.0978 - val_loss: 3.5085 - val_accuracy: 0.1662
Epoch 2/4
1407/1407 [==============================] - 105s 74ms/step - loss: 3.2653 - accuracy: 0.2126 - val_loss: 3.0062 - val_accuracy: 0.2622
Epoch 3/4
1407/1407 [==============================] - 104s 74ms/step - loss: 2.8041 - accuracy: 0.3001 - val_loss: 2.7466 - val_accuracy: 0.3160
Epoch 4/4
1407/1407 [==============================] - 105s 75ms/step - loss: 2.4589 - accuracy: 0.3685 - val_loss: 2.6118 - val_accuracy: 0.3406


In [8]:
test_loss, test_acc = conv_model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.4f}")

313/313 [==============================] - 8s 26ms/step - loss: 2.6058 - accuracy: 0.3449
Test accuracy: 0.3449
